### 模型的泛化能力：预测准确性 (predictive accuracy)

前面提到的诸多模型拟合优度 (Goodness of fit) 指标，只能衡量模型对于当前样本的拟合程度。

对于样本外的数据，我们不确定该模型是否具有**泛化能力**，即该模型是否能准确的预测样本外的数据。

为了评估模型的**预测准确性 (predictive accuracy)**，我们有以下3种策略：
1. 通过新数据对模型进行评估。我们可以收集新的数据，并检验模型的预测能力。
2. 从已有样本中拿出一部分数据用来预测。即交叉验证。
3. 通过统计方法进行近似。比如使用对交叉验证近似的信息熵指标来评估模型的泛化能力。

#### 交叉验证

收集新数据来检验模型的预测能力是一种理所当然的直觉。但心理学数据不同于其他学科的数据，它常受到**时间因素**和**抽样**的影响。
- 比如，心境可能随着季节变化，因此在不同季节收集到的数据会受到时间的影响。

因此，一种更高效的方法是，一次性多收集一些数据，选择其中的一部分作为预测数据。

但问题在于，我们选择哪一部分数据作为预测数据呐？或者说，我们该如何有效的对数据进行抽取呐？

**交叉验证**的目的就在于：提供不同的抽取预测数据的策略

常见的交叉验证策略：
1. 分半交叉验证 (Split-half cross-validation)
	- 分半交叉验证将观测数据对半分成两部分，分别在不同的数据集上拟合模型，并在另外一半数据集上验证模型，最后再对比不同的模型在两份数据集作为验证集时的预测准确度。
2. K 折交叉验证 (K-fold cross-validation) 
	- K 折交叉验证把数据分成 K 分，其中一份作为训练集，其余的 K-1 分数据集作为验证集，总共重复这个流程 K 次。以 K 次验证结果的均值作为验证标准。
3. 留一法交叉验证 (Leave-one-out cross-validation)
	- 留一法交叉验证是 K 折交叉验证的一个特例，当分折的数量等于数据的数量时，K 折留一法便成了留一法交叉验证。留一法交叉验证相较于普通的交叉验证方法，几乎使用了所有数据去训练模型，因此留一法交叉验证的训练模型时的**偏差 (bias) 更小、更鲁棒**，但是又因为验证集只有一个数据点，验证模型的时候**留一法交叉验证的方差 (Variance) 也会更大**。

K 折交叉验证图示
![](https://hub.packtpub.com/wp-content/uploads/2019/05/KFold.png)

source: https://hub.packtpub.com/cross-validation-strategies-for-time-series-forecasting-tutorial/

#### 信息准则(information criteria)

尽管交叉验证存在诸多优势，并且避免了收集数据的潜在问题，但是交叉验证**在认知神经科学里的认知建模领域里的使用并不广泛**，

其最主要的原因在于，研究者收集的数据**样本量往往有限**，然而很多计算模型却对数据样本量有所需求，如果使用交叉验证，将数据拆分，那么很有可能导致拟合模型的试次数量不足，使得模型拟合和验证的结果较差，进而导致产生一类错误和二类错误的概率增大。

为了解决这些问题，统计学家提出了一种衡量预测准确性的指标，即**信息准则**(information criteria)。

信息准则是对统计模型**预测精度**的一种度量。它考虑**模型与数据的匹配程度**，并通过**模型的复杂性**进行矫正(correction to bias)。

$信息准则 = deviance + correction$
- deviance 为偏差，反应了模型与数据的匹配程度。可以通过对数似然 log likelihood 进行计算。注意 log likelihood 也可以称为 lpd (log predictive density)。
- correction 为矫正，与模型的复杂程度相关。模型越复杂时越容易过拟合(overfitting)，因此，矫正项也会越大。

可见，信息准则越小，偏差和矫正就越小。因此，**信息准则越小，代表模型的预测性越好**。

常见信息准则有4类：
1. AIC (Akaike information criterion)
2. DIC (Deviance information criterion)
3. WAIC (Widely applicable information criterion)
4. LOO-CV (Leave-one-out cross-validation)

#### 1. AIC (Akaike information criterion)

AIC是最简单的信息准则标准，由日本统计学家赤池弘次 (Hirotugu Akaike) 提出 (Akaike, 1974)，是频率主义统计学里最为经典的模型比较指标之一。

其表达式如下：
$$
A I C= -2 \sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)+2 p_{A I C}
$$


$-2 \sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)$ 为偏差deviance，描述了模型对于当前数据的匹配程度。
- 其中，$\hat{\theta}_{m l e}$为最大似然法求得的参数值。 
- $\log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)$ 为对数似然(log likelihood)，也可称为 lpd (log predictive density)。
- 偏差deviance为负二倍(-2*)的对数似然值。模型拟合的越好，似然值越大，因此$\sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)$越大。相应的 $-2\sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)$ 的值越小。那么 AIC的值也越小。

$2 p_{A I C}$ 为矫正(correction)。
- 其中，$p_{A I C}$ 为模型的参数数量。
- 矫正(correction)为参数数量的两倍，描述了模型的复杂程度。模型越复杂，潜在的参数数量可能越多，那么$p_{A I C}$越大。

需要注意的是 AIC 只考虑了最大似然对应的参数值 $\hat{\theta}_{m l e}$, 因此它适用于频率学派模型的评估。而对于贝叶斯学派来说，**由于参数为分布，因此不能使用AIC来评估模型**。

#### 2. DIC (Deviance information criterion)

为了解决 AIC 无法评估贝叶斯模型。

统计学家们提出了"贝叶斯参数估计版的 AIC"，即 DIC (Deviance information criterion) 。

$$
\mathrm{DIC} = -2 \sum_{i}^{n} \log p\left(y_{i} \mid \bar{\theta}\right) +2 p_{DIC}
$$

$-2 \sum_{i}^{n} \log p\left(y_{i} \mid \bar{\theta}\right)$ 为偏差(deviance)，记为D。
- 可以看到，该偏差与AIC中的偏差$-2 \sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)$非常相似。
- 区别在于，AIC 中的 $\hat{\theta}_{m l e}$被替换为$\bar{\theta}$。因为贝叶斯框架中，参数不是固定值而是概率分布，因此 $\bar{\theta}$ 代表的是参数后验分布的均值 $\bar{\theta}$。

![Image Name](https://cdn.kesci.com/upload/image/rl7ys1gvzz.png?imageView2/0/w/640/h/640)

$p_{\mathrm{DIC}}$ 为矫正(correction). 

$p_{\mathrm{DIC}}= 2(\sum_{i}^{n} \log p\left(y_{i} \mid \bar{\theta}\right)-\frac{1}{S} \sum_{s=1}^{S} \log p\left(y \mid \theta^{s}\right))$
- 其中$\sum_{i}^{n} \log p\left(y_{i} \mid \bar{\theta}\right)$为偏差D。
- $\frac{1}{S} \sum_{s=1}^{S} \log p\left(y \mid \theta^{s}\right)$为所有参数采样所计算到的对数似然的平均值。参数采样包含s个样本，编号为1到s。

相比于AIC的矫正项 $p_{\mathrm{AIC}}$，$p_{\mathrm{DIC}}$ 是基于数据的矫正(data-based bias correction)。因为 $p_{\mathrm{DIC}}$ 考虑了数据在不同参数的对数似然下的影响。

$p_{\mathrm{DIC}}$ 存在另一种表达方式：
$$
p_{\mathrm{DIC}}=2\operatorname{var}(\log p(y \mid \theta))
$$
- 其中，var表示计算方差。$\operatorname{var}(\log p(y \mid \theta))$表示所有参数计算所得到的对数似然的方差。
- 这种计算方式与第一种计算方式得到的结果一致，并且可以避免 $p_{\mathrm{DIC}}$ 为负数。

DIC  是心理学领域最常用的模型评估指标之一。
- 与 AIC 只是简单地使用了参数数量作为复杂度的惩罚项不同的是，DIC **利用了 MCMC 采样的参数后验分布去计算模型的有效参数数量**。
- 另外 DIC的计算速度与AIC一样很快，这与后面会介绍的其他指标形成对比。
- 最后，除了使用参数分布的均值去计算 DIC 中 bias，也可以**使用参数分布的中位数等计算 bias，这提高了 DIC 计算的灵活性**。

#### 3. WAIC (Widely applicable information criterion)

WAIC (Widely applicable information criterion) 为“DIC 的升级版”, 由日本统计学家渡边澄夫提出。

相较于 DIC 只考虑了在参数后验分布对数似然的均值，WAIC 考虑了每一个数据点在不同后验参数的影响。

$$
W A I C=-2\sum_{i}^{n} \log \left(\frac{1}{s} \sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right)\right)+2p_{\mathrm{WAIC}}
$$

![Image Name](https://cdn.kesci.com/upload/image/rl7ysdhz49.png?imageView2/0/w/640/h/640)

$-2\sum_{i}^{n} \log \left(\frac{1}{s} \sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right)\right)$ 为偏差(deviance)。

- WAIC 的偏差与 DIC的偏差 $-2 \sum_{i}^{n} \log p\left(y_{i} \mid \bar{\theta}\right)$ 相似，
- WAIC 与 DIC 的区别在于增加了 $\frac{1}{s}\sum_{j}^{S}$, 代表每个数据点在不同后验参数下似然值的均值。
- 因为这个特性，WAIC 的偏差有个新名字，lppd (log pointwise predictive density)。
- lppd 与前面的偏差 (lpd, log predictive density)的区别在于，多了pointwise的步骤：
  - WAIC 每次选择一个数据点，计算它在所有后验采样上的似然值 $\sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right)$，
  - 再求这些似然值在不同后验参数$\theta^j$上的平均值 $\frac{1}{s} \sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right)$，
  - 最后将不同数据点上的似然值求和，即 $\sum_{i}^{n} \log \left(\frac{1}{s} \sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right)\right) = \sum_{i}^{n} \log \bar{L}$。

与 DIC 类似的是，矫正项 $p_{\mathrm{WAIC}}$ 有两种表达形式：

- $p_{\mathrm{WAIC}}=2 \sum_{i=1}^{n}\left(\log \left(\frac{1}{S} \sum_{s=1}^{S} p\left(y_{i} \mid \theta^{s}\right)\right)-\frac{1}{S} \sum_{s=1}^{S} \log p\left(y_{i} \mid \theta^{s}\right)\right)$
- $p_{\mathrm{WAIC}} = \sum_{i}^{n}\left({V}^{s}_{j} \log p\left(Y_{i} \mid \boldsymbol{\theta}^{j}\right)\right)$

其中 $\sum_{i=1}^{n}\log \left(\frac{1}{S} \sum_{s=1}^{S} p\left(y_{i} \mid \theta^{s}\right)\right)$ 为偏差或 lppd。 $\frac{1}{S} \sum_{s=1}^{S} \log p\left(y_{i} \mid \theta^{s}\right)$ 为单个数据点$y_{i}$在所有后验参数$\theta^{s}$下对数似然的均值。

#### 4. LOO-CV (Leave-one-out cross-validation)

WAIC 是非常优秀且常用的指标，其本质是模型对于未知数据的预测能力的**近似**。

另一种与 WAIC 非常类似的近似方法是前文提到的**留一交叉验证法** (Leave-one-out cross-validation, LOO-CV) 

$$
ELPD_{LOO-CV} = \sum_{i}^{n} \log \left(\frac{1}{s} \sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}_{-i}^{j}\right)\right)
$$

- ELPD 为 expected log predictive density。
- $ELPD_{LOO-CV}$ 利用了留一交叉验证的思想，用去除数据点i剩下的数据$y_-i$拟合模型；再回过来用该模型的参数去预测数据点$y_i$。
- 因此，ELPD 与 WAIC 的偏差非常类似。区别在于，似然中的参数值不是通过所有数据进行拟合的，而是通过去除数据点i剩下的数据拟合得到的参数 $\theta_{-i}$。
- 此外，更巧妙的是，由于 $ELPD_{LOO-CV}$ 是直接对非拟合数据$y_i$进行预测，因此不需要再矫正模型。

在实际操作中，我们通过 `ArViz` 的函数可以很容易的获得 WAIC 和 $ELPD_{LOO-CV}$，我们称为 **LOO** 方法。

此外，由于 $ELPD_{LOO-CV}$ 的计算量也比较大，ArViz 会使用 Pareto Smooth Importance Sampling Leave Once Out Cross Validation (PSIS-LOO-CV) 来近似 $ELPD_{LOO-CV}$。

PSIS-LOO-CV 有两大优势：
1. 计算速度快，且结果稳健
2. 提供了丰富的模型诊断指标

### Model Averaging

前面我们讨论了如何评估单个模型的**拟合优度**与**预测精度**。

但判断模型预测性能的另一个思路是：拟合多个模型，比较不同模型的预测能力。在实践中，我们对不同的模型赋予不同的权重，并组合他们生成一个元模型 (meta-model)，进而进行元预测，以此评估不同模型权重的影响。

这种给不同模型赋予权重的方法称为 模型平均法 Model Averaging。
常见有三种计算模型权重的方式：
1. Marginal likelihood
2. BIC (Bayesian information criterion)
3. Pseudo Bayesian model averaging

#### 1. Marginal likelihood

边缘似然或者边际似然 (marginal likelihood) 是贝叶斯公式的分布部分
- 即 $p(\theta|data)=\frac{p(data|\theta)p(\theta)}{p(data)}$ 中的 $p(data) = \int_{\theta}^{} p(data|\theta)p(\theta)d\theta$ 
- 边缘似然与贝叶斯公式分子部分的似然不同，表达了模型对数据的平均拟合 (Average fit)，因此它可以作为模型选择的指标。*边缘似然越大，说明模型对样本数据解释的越好*。
- 当比较两个模型时，可以将边缘似然转化为**贝叶斯因子**(Bayes factor)。

Bayesian Model Averaging

$$
w_k = \frac{e^{-ML_{k}}}{\sum e^{-ML_{k}}}
$$

通过边缘似然可以计算 Bayesian Model Averaging。
- 假设有 k 各模型。
- k个模型的边缘似然为 $ML_{k}$。
- k个模型的权重 $w_k$ 为当前模型的边缘似然 $ML_{k}$ 比上 所有模型边缘似然之和 $\sum e^{-ML_{k}}$。


#### 2. BIC (Bayesian information criterion)

因为边缘似然计算量巨大，因此我们需要一些快速的计算方式，比如 BIC (Bayesian information criterion)。

$$
A I C=-2 \sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)+2 k
$$

$$
B I C=-2 \sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right) + 2 k*ln(n)
$$

BIC 与 AIC 非常类似，区别在于惩罚项 error的不同。
- BIC 中的 error 在 AIC error 的基础上乘以 ln(n)。
- 其中，k为模型参数的数量；n为数据的数量。

BIC 的特点：
- 在公式上与 AIC 高度相似，因此可以用来检验模型拟合优度，其值越小，模型拟合越好。
- BIC 的 error 往往比 AIC 的更大，即惩罚更大，因此，**BIC 通常会选择简单的模型**。
- BIC 虽然适用于贝叶斯模型，但是它没有考虑先验的影响。
- 最重要的是，BIC **是边缘似然的近似**，计算速度比 ML 更快，并且同样也可以被用来计算贝叶斯因子。

#### 3. Pseudo Bayesian model averaging

BIC 虽然能近似边缘似然， 但是其 error 只考虑了参数数量和数据数量的复杂性，也没有考虑到先验的影响。
这样计算的模型权重很可能存在偏差。

为了更高效的计算模型的权重。一种可行的方法是 Pseudo Bayesian model averaging。
- 即通过 WAIC 与 LOO 来近似边缘似然。
- 再通过 Bayesian model averaging 公式计算模型权重。


$$
w_i = \frac{e^{-\Delta_{i}}}{\sum_{j}^{k} e^{-\Delta_{j}}}
$$


上面的公式与 Bayesian model averaging 的公式 $w_k = \frac{e^{-ML_{k}}}{\sum e^{-ML_{k}}}$ 很像。
区别在于：
- 通过 WAIC 或者 LOO 在模型中的差值 $\Delta_{j}$ 替代了 边缘似然 ML。
- $\Delta$ 表示的是，第 i 或者 j个模型的 WAIC 与 最优模型的 WAIC 的差值。

PyMC3 与 Arviz 提高了很多关于 Pseudo Bayesian model averaging 计算的方法，之后的实践中，我们将注重于通过 Pseudo Bayesian model averaging 展示模型平均法的作用。

总结：

模型拟合优度的方法包括：
- 拟合优度 
- mse 
- 对数似然

模型预测进度的方法包括：
- AIC
- DIC
- WAIC
- LOO

模型平均法包括：
- Bayesian model averaging
- BIC
- Pseudo Bayesian model averaging

|                    | AIC                                  | DIC                                      | WAIC       | LOOCV           | BIC                                  |
| ------------------ | ------------------------------------ | ---------------------------------------- | ---------- | --------------- | ------------------------------------ |
| 适用框架           | 频率论                               | 贝叶斯                                   | 贝叶斯     | 贝叶斯          | 贝叶斯/频率论                        |
| 偏差（deviance）   | 最大似然参数 $\theta_mle$ 的对数似然 | 贝叶斯参数均值 $\bar{\theta}$ 的对数似然 | LPPD       | $ELPD_{LOO-CV}$ | 最大似然参数 $\theta_mle$ 的对数似然 |
| 矫正（correction） | 参数数量                             | 似然的变异                               | 似然的变异 |     由于采用 LOO-CV 思想，因此不需要矫正            | 参数数量+数据数量                    |